# Compressor

Simple script to compress images and videos.

In [ ]:
import os

# Add the ffmpeg file from your binary into the `IMAGEIO_FFMPEG_EXE` environment variable.
# You might try to run the notebook without this line and see if it works.
os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

from moviepy.editor import VideoFileClip
from PIL import Image

# Edit the following variables to meet your requirements. This is the only thing you must change.

IMAGE_TARGET_SIZE = 0.2 * 1024 * 1024   # Just change the 0.2 to the desired size in MB.
VIDEO_TARGET_SIZE = 10 * 1024 * 1024    # Just change the 10 to the desired size in MB.

# Set the maximum resolution (e.g., 1920x1080) for both images and videos.
MAX_WIDTH = 1920
MAX_HEIGHT = 1080


## Images

The following script compresses images using the `Pillow` library. It reads all images from the input directory, compresses them to the desired quality, and saves them in `results` inside the input directory.

In [ ]:
def resize_image(img):
    width, height = img.size
    if width > MAX_WIDTH or height > MAX_HEIGHT:
        # Calculate the new size preserving the aspect ratio
        ratio = min(MAX_WIDTH / width, MAX_HEIGHT / height)
        new_size = (int(width * ratio), int(height * ratio))
        img = img.resize(new_size, Image.LANCZOS)
        print(f"Image resized to: {new_size}")
    return img

def compress_image(file_path):
    print(f"\nCompressing {file_path}...")
    img = Image.open(file_path)

    # Create the results directory if it doesn't exist
    results_dir = os.path.join(os.path.dirname(file_path), 'results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Add result file path inside results folder
    result_file_path = os.path.join(results_dir, os.path.basename(file_path))
    
    # Check the size of the original image
    original_size = os.path.getsize(file_path)
    if original_size <= IMAGE_TARGET_SIZE:
        print("Image is already within the target size.")
        return

    # Convert RGBA to RGB if needed
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    # Resize image if it's too large
    img = resize_image(img)

    quality = 95  # Start with high quality
    while True:
        # Save the image with the current quality
        img.save("temp.jpg", quality=quality)
        # Check the size
        size = os.path.getsize("temp.jpg")
        if size <= IMAGE_TARGET_SIZE or quality <= 10:
            print("Break loop with quality: ", quality)
            # If the size is okay or quality is too low, stop the loop
            break
        # Reduce quality for the next iteration
        quality -= 5

    # Save the compressed file in the results directory at the original file path
    img.save(result_file_path, quality=quality)
    os.remove("temp.jpg")

def compress_images_in_directory(directory_path):
    # Iterate over all files in the specified directory
    for file_name in os.listdir(directory_path):
        try:
            if file_name.lower().endswith('.jpg'):
                file_path = os.path.join(directory_path, file_name)
                compress_image(file_path)
        except Exception as e:
            print(f"An error occurred: {e}")
    print("Compression complete.\n")

# Specify the complete path to the directory
directory_path = '/Users/username/directory'
compress_images_in_directory(directory_path)


## Videos

The following script compresses videos using the `moviepy` library. It reads all videos from the input directory, compresses them to the desired bitrate, and saves them in `results` inside the input directory.

In [ ]:
def resize_video(file_path):
    print(f"\nProcessing {file_path}...")
    clip = VideoFileClip(file_path)

    # Create the results directory if it doesn't exist
    results_dir = os.path.join(os.path.dirname(file_path), 'results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Add result file path inside results folder
    result_file_path = os.path.join(results_dir, os.path.basename(file_path))
    
    # Calculate the new size preserving the aspect ratio
    width, height = clip.size
    if width > MAX_WIDTH or height > MAX_HEIGHT:
        ratio = min(MAX_WIDTH / width, MAX_HEIGHT / height)
        new_size = (int(width * ratio), int(height * ratio))
        clip = clip.resize(new_size)
        print(f"Video resized to: {new_size}")
    
    # Compress video by reducing bitrate
    temp_output = "temp.mp4"
    bitrate = '5000k'  # Starting bitrate
    while True:
        print("Trying bitrate:", bitrate)
        clip.write_videofile(temp_output, bitrate=bitrate, audio=True)
        print("Completed writing")
        size = os.path.getsize(temp_output)
        print(f"Bitrate: {bitrate}, Size: {size}")
        if size <= VIDEO_TARGET_SIZE or int(bitrate[:-1]) <= 500:
            print("Break loop with bitrate: ", bitrate)
            break
        bitrate = f"{int(bitrate[:-1]) - 500}k"
    
    # Save the compressed file in the results directory at the original file path
    os.replace(temp_output, result_file_path)

def compress_videos_in_directory(directory_path):
    for file_name in os.listdir(directory_path):
        try:
            if file_name.lower().endswith('.mp4'):
                file_path = os.path.join(directory_path, file_name)
                resize_video(file_path)
        except Exception as e:
            print(f"An error occurred: {e}")
    print("Compression complete.\n")

# Specify the complete path to the directory
directory_path = '/Users/username/directory'
compress_videos_in_directory(directory_path)